In [1]:
import pandas as pd
import numpy as np
import re
import time
from textblob import TextBlob
import sqlite3
import GetOldTweets
import datetime

# Data Preprocessing

In [2]:
database = 'database.sqlite'
conn = sqlite3.connect(database)

In [3]:
#Fetching required data tables
player_data = pd.read_sql("SELECT * FROM Player;", conn)
player_stats_data = pd.read_sql("SELECT * FROM Player_Attributes;", conn)
team_data = pd.read_sql("SELECT * FROM Team;", conn)
match_data = pd.read_sql("SELECT * FROM Match;", conn)
countries = pd.read_sql_query("SELECT * from Country", conn)
leagues = pd.read_sql_query("SELECT * from League", conn)
team_data.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [4]:
# Initializing search parameter data
search_params = pd.DataFrame(columns=['match_api_id', 'date', 'start_date', 'end_date', 'home_team_api_id', 'away_team_api_id', 'home_team_name', 'away_team_name', 'home_team_positive_tweets', 'home_team_neutral_tweets','home_team_negative_tweets','away_team_positive_tweets','away_team_neutral_tweets','away_team_negative_tweets'])
search_params['match_api_id'] = match_data['match_api_id']
search_params['date'] = match_data['date']
search_params['home_team_api_id'] = match_data['home_team_api_id']
search_params['away_team_api_id'] = match_data['away_team_api_id']
search_params.head()

,match_api_id,date,start_date,end_date,home_team_api_id,away_team_api_id,home_team_name,away_team_name,home_team_positive_tweets,home_team_neutral_tweets,home_team_negative_tweets,away_team_positive_tweets,away_team_neutral_tweets,away_team_negative_tweets
0,492473,2008-08-17 00:00:00,NaN,NaN,9987,9993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,492474,2008-08-16 00:00:00,NaN,NaN,10000,9994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,492475,2008-08-16 00:00:00,NaN,NaN,9984,8635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,492476,2008-08-17 00:00:00,NaN,NaN,9991,9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,492477,2008-08-16 00:00:00,NaN,NaN,7947,9985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Getting start_date and end_date for search_params
for i in range(0, len(search_params)):
    val = search_params.iloc[i, search_params.columns.get_loc('date')]
    
    # getting first token of date
    date = val.split(' ', 1)[0]
    search_params.iloc[i, search_params.columns.get_loc('date')] = date
    
    # parsing date into year, month, day
    year = date.split('-', 2)[0]
    year = year[2:]
    month = date.split('-', 2)[1]
    day = date.split('-', 2)[2]
    
    # converting date to datetime type
    match_date = datetime.datetime.strptime(year + "-" + month + "-" + day, "%y-%m-%d")
    
    # subtracting 10 days from match_date to get start_date
    start_date = match_date + datetime.timedelta(days=-10)
    search_params.iloc[i, search_params.columns.get_loc('start_date')] = str(start_date).split(' ', 1)[0]
    
    # subtracting 1 day from match_date to get end_date
    end_date = match_date + datetime.timedelta(days=-1)
    search_params.iloc[i, search_params.columns.get_loc('end_date')] = str(end_date).split(' ', 1)[0]
    

In [6]:
search_params.head()

,match_api_id,date,start_date,end_date,home_team_api_id,away_team_api_id,home_team_name,away_team_name,home_team_positive_tweets,home_team_neutral_tweets,home_team_negative_tweets,away_team_positive_tweets,away_team_neutral_tweets,away_team_negative_tweets
0,492473,2008-08-17,2008-08-07,2008-08-16,9987,9993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,492474,2008-08-16,2008-08-06,2008-08-15,10000,9994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,492475,2008-08-16,2008-08-06,2008-08-15,9984,8635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,492476,2008-08-17,2008-08-07,2008-08-16,9991,9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,492477,2008-08-16,2008-08-06,2008-08-15,7947,9985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Getting home_team_name and away_team name for search_params
for i in range(0, len(search_params)):   
    # Getting team ids
    home_team_api_id = search_params.iloc[i, search_params.columns.get_loc('home_team_api_id')]
    away_team_api_id = search_params.iloc[i, search_params.columns.get_loc('away_team_api_id')]
    
    # Getting index of ids in team_data
    home_team_index = team_data.index[team_data['team_api_id'] == home_team_api_id]
    away_team_index = team_data.index[team_data['team_api_id'] == away_team_api_id]
    
    # Getting team names
    home_team_name = team_data.iloc[home_team_index, team_data.columns.get_loc('team_long_name')].values[0]
    away_team_name = team_data.iloc[away_team_index, team_data.columns.get_loc('team_long_name')].values[0]
    
    search_params.iloc[i, search_params.columns.get_loc('home_team_name')] = home_team_name
    search_params.iloc[i, search_params.columns.get_loc('away_team_name')] = away_team_name

In [8]:
search_params.head()

,match_api_id,date,start_date,end_date,home_team_api_id,away_team_api_id,home_team_name,away_team_name,home_team_positive_tweets,home_team_neutral_tweets,home_team_negative_tweets,away_team_positive_tweets,away_team_neutral_tweets,away_team_negative_tweets
0,492473,2008-08-17,2008-08-07,2008-08-16,9987,9993,KRC Genk,Beerschot AC,NaN,NaN,NaN,NaN,NaN,NaN
1,492474,2008-08-16,2008-08-06,2008-08-15,10000,9994,SV Zulte-Waregem,Sporting Lokeren,NaN,NaN,NaN,NaN,NaN,NaN
2,492475,2008-08-16,2008-08-06,2008-08-15,9984,8635,KSV Cercle Brugge,RSC Anderlecht,NaN,NaN,NaN,NaN,NaN,NaN
3,492476,2008-08-17,2008-08-07,2008-08-16,9991,9998,KAA Gent,RAEC Mons,NaN,NaN,NaN,NaN,NaN,NaN
4,492477,2008-08-16,2008-08-06,2008-08-15,7947,9985,FCV Dender EH,Standard de Liège,NaN,NaN,NaN,NaN,NaN,NaN


# Twitter Sentiment Analysis

In [2]:
search_params = pd.read_csv("Twitter_NLP.csv")

In [3]:
search_params.iloc[3500, search_params.columns.get_loc('home_team_positive_tweets')]

0.28000000000000003

In [6]:
api = TwitterClient()
count = 0
for i in range(3501, len(search_params)):
    try:
        tweets = api.get_tweets(
            query=search_params.iloc[i, search_params.columns.get_loc('home_team_name')],
            since=search_params.iloc[i, search_params.columns.get_loc('start_date')],
            until=search_params.iloc[i, search_params.columns.get_loc('end_date')],
            count=100)
        if (len(tweets) != 0):
            positive, negative, neutral = get_sentiment(tweets)
        search_params.iloc[i, search_params.columns.get_loc('home_team_positive_tweets')] = positive
        search_params.iloc[i, search_params.columns.get_loc('home_team_negative_tweets')] = negative
        search_params.iloc[i, search_params.columns.get_loc('home_team_neutral_tweets')] = neutral
        
        tweets = api.get_tweets(
            query=search_params.iloc[i, search_params.columns.get_loc('away_team_name')],
            since=search_params.iloc[i, search_params.columns.get_loc('start_date')],
            until=search_params.iloc[i, search_params.columns.get_loc('end_date')],
            count=100)
        if (len(tweets) != 0):
            positive, negative, neutral = get_sentiment(tweets)
        search_params.iloc[i, search_params.columns.get_loc('away_team_positive_tweets')] = positive
        search_params.iloc[i, search_params.columns.get_loc('away_team_negative_tweets')] = negative
        search_params.iloc[i, search_params.columns.get_loc('away_team_neutral_tweets')] = neutral
        print ("Percentage complete: {0:.2f}".format(i / 25979. * 100.))
        if (i % 100 == 0):
            search_params.to_csv("Twitter_NLP.csv", encoding='utf-8')
    except:
        print ("ERROR")

Percentage complete: 13.48
Percentage complete: 13.48
Percentage complete: 13.48
Percentage complete: 13.49
Percentage complete: 13.49
Percentage complete: 13.50
Percentage complete: 13.50
Percentage complete: 13.50
Percentage complete: 13.51
Percentage complete: 13.51
Percentage complete: 13.51
Percentage complete: 13.52
Percentage complete: 13.52
Percentage complete: 13.53
Percentage complete: 13.53
Percentage complete: 13.53
Percentage complete: 13.54
Percentage complete: 13.54
Percentage complete: 13.55
Percentage complete: 13.55
Percentage complete: 13.55
Percentage complete: 13.56
Percentage complete: 13.56
Percentage complete: 13.56
Percentage complete: 13.57
Percentage complete: 13.57
Percentage complete: 13.58
Percentage complete: 13.58
Percentage complete: 13.58
Percentage complete: 13.59
Percentage complete: 13.59
Percentage complete: 13.60
Percentage complete: 13.60
Percentage complete: 13.60
Percentage complete: 13.61
Percentage complete: 13.61
Percentage complete: 13.61
P

Percentage complete: 14.65
Percentage complete: 14.65
Percentage complete: 14.65
Percentage complete: 14.66
Percentage complete: 14.66
Percentage complete: 14.67
Percentage complete: 14.67
Percentage complete: 14.67
Percentage complete: 14.68
Percentage complete: 14.68
Percentage complete: 14.68
Percentage complete: 14.69
Percentage complete: 14.69
Percentage complete: 14.70
Percentage complete: 14.70
Percentage complete: 14.70
Percentage complete: 14.71
Percentage complete: 14.71
Percentage complete: 14.72
Percentage complete: 14.72
Percentage complete: 14.72
Percentage complete: 14.73
Percentage complete: 14.73
Percentage complete: 14.73
Percentage complete: 14.74
Percentage complete: 14.74
Percentage complete: 14.75
Percentage complete: 14.75
Percentage complete: 14.75
Percentage complete: 14.76
Percentage complete: 14.76
Percentage complete: 14.77
Percentage complete: 14.77
Percentage complete: 14.77
Percentage complete: 14.78
Percentage complete: 14.78
Percentage complete: 14.79
P

Percentage complete: 15.82
Percentage complete: 15.82
Percentage complete: 15.82
Percentage complete: 15.83
Percentage complete: 15.83
Percentage complete: 15.84
Percentage complete: 15.84
Percentage complete: 15.84
Percentage complete: 15.85
Percentage complete: 15.85
Percentage complete: 15.86
Percentage complete: 15.86
Percentage complete: 15.86
Percentage complete: 15.87
Percentage complete: 15.87
Percentage complete: 15.87
Percentage complete: 15.88
Percentage complete: 15.88
Percentage complete: 15.89
Percentage complete: 15.89
Percentage complete: 15.89
Percentage complete: 15.90
Percentage complete: 15.90
Percentage complete: 15.91
Percentage complete: 15.91
Percentage complete: 15.91
Percentage complete: 15.92
Percentage complete: 15.92
Percentage complete: 15.92
Percentage complete: 15.93
Percentage complete: 15.93
Percentage complete: 15.94
Percentage complete: 15.94
Percentage complete: 15.94
Percentage complete: 15.95
Percentage complete: 15.95
Percentage complete: 15.96
P

Percentage complete: 16.99
Percentage complete: 16.99
Percentage complete: 16.99
Percentage complete: 17.00
Percentage complete: 17.00
Percentage complete: 17.01
Percentage complete: 17.01
Percentage complete: 17.01
Percentage complete: 17.02
Percentage complete: 17.02
Percentage complete: 17.03
Percentage complete: 17.03
Percentage complete: 17.03
Percentage complete: 17.04
Percentage complete: 17.04
Percentage complete: 17.04
Percentage complete: 17.05
Percentage complete: 17.05
Percentage complete: 17.06
Percentage complete: 17.06
Percentage complete: 17.06
Percentage complete: 17.07
Percentage complete: 17.07
Percentage complete: 17.08
Percentage complete: 17.08
Percentage complete: 17.08
Percentage complete: 17.09
Percentage complete: 17.09
Percentage complete: 17.09
Percentage complete: 17.10
Percentage complete: 17.10
Percentage complete: 17.11
Percentage complete: 17.11
Percentage complete: 17.11
Percentage complete: 17.12
Percentage complete: 17.12
Percentage complete: 17.13
P

Percentage complete: 18.16
Percentage complete: 18.16
Percentage complete: 18.16
Percentage complete: 18.17
Percentage complete: 18.17
Percentage complete: 18.18
Percentage complete: 18.18
Percentage complete: 18.18
Percentage complete: 18.19
Percentage complete: 18.19
Percentage complete: 18.20
Percentage complete: 18.20
Percentage complete: 18.20
Percentage complete: 18.21
Percentage complete: 18.21
Percentage complete: 18.21
Percentage complete: 18.22
Percentage complete: 18.22
Percentage complete: 18.23
Percentage complete: 18.23
Percentage complete: 18.23
Percentage complete: 18.24
Percentage complete: 18.24
Percentage complete: 18.25
Percentage complete: 18.25
Percentage complete: 18.25
Percentage complete: 18.26
Percentage complete: 18.26
Percentage complete: 18.26
Percentage complete: 18.27
Percentage complete: 18.27
Percentage complete: 18.28
Percentage complete: 18.28
Percentage complete: 18.28
Percentage complete: 18.29
Percentage complete: 18.29
Percentage complete: 18.30
P

...

Percentage complete: 94.20
Percentage complete: 94.20
Percentage complete: 94.21
Percentage complete: 94.21
Percentage complete: 94.21
Percentage complete: 94.22
Percentage complete: 94.22
Percentage complete: 94.23
Percentage complete: 94.23
Percentage complete: 94.23
Percentage complete: 94.24
Percentage complete: 94.24
Percentage complete: 94.25
Percentage complete: 94.25
Percentage complete: 94.25
Percentage complete: 94.26
Percentage complete: 94.26
Percentage complete: 94.26
Percentage complete: 94.27
Percentage complete: 94.27
Percentage complete: 94.28
Percentage complete: 94.28
Percentage complete: 94.28
Percentage complete: 94.29
Percentage complete: 94.29
Percentage complete: 94.30
Percentage complete: 94.30
Percentage complete: 94.30
Percentage complete: 94.31
Percentage complete: 94.31
Percentage complete: 94.31
Percentage complete: 94.32
Percentage complete: 94.32
Percentage complete: 94.33
Percentage complete: 94.33
Percentage complete: 94.33
Percentage complete: 94.34
P

Percentage complete: 95.37
Percentage complete: 95.37
Percentage complete: 95.38
Percentage complete: 95.38
Percentage complete: 95.38
Percentage complete: 95.39
Percentage complete: 95.39
Percentage complete: 95.40
Percentage complete: 95.40
Percentage complete: 95.40
Percentage complete: 95.41
Percentage complete: 95.41
Percentage complete: 95.42
Percentage complete: 95.42
Percentage complete: 95.42
Percentage complete: 95.43
Percentage complete: 95.43
Percentage complete: 95.43
Percentage complete: 95.44
Percentage complete: 95.44
Percentage complete: 95.45
Percentage complete: 95.45
Percentage complete: 95.45
Percentage complete: 95.46
Percentage complete: 95.46
Percentage complete: 95.47
Percentage complete: 95.47
Percentage complete: 95.47
Percentage complete: 95.48
Percentage complete: 95.48
Percentage complete: 95.48
Percentage complete: 95.49
Percentage complete: 95.49
Percentage complete: 95.50
Percentage complete: 95.50
Percentage complete: 95.50
Percentage complete: 95.51
P

Percentage complete: 96.54
Percentage complete: 96.54
Percentage complete: 96.55
Percentage complete: 96.55
Percentage complete: 96.55
Percentage complete: 96.56
Percentage complete: 96.56
Percentage complete: 96.57
Percentage complete: 96.57
Percentage complete: 96.57
Percentage complete: 96.58
Percentage complete: 96.58
Percentage complete: 96.59
Percentage complete: 96.59
Percentage complete: 96.59
Percentage complete: 96.60
Percentage complete: 96.60
Percentage complete: 96.60
Percentage complete: 96.61
Percentage complete: 96.61
Percentage complete: 96.62
Percentage complete: 96.62
Percentage complete: 96.62
Percentage complete: 96.63
Percentage complete: 96.63
Percentage complete: 96.64
Percentage complete: 96.64
Percentage complete: 96.64
Percentage complete: 96.65
Percentage complete: 96.65
Percentage complete: 96.65
Percentage complete: 96.66
Percentage complete: 96.66
Percentage complete: 96.67
Percentage complete: 96.67
Percentage complete: 96.67
Percentage complete: 96.68
P

Percentage complete: 97.71
Percentage complete: 97.71
Percentage complete: 97.72
Percentage complete: 97.72
Percentage complete: 97.73
Percentage complete: 97.73
Percentage complete: 97.73
Percentage complete: 97.74
Percentage complete: 97.74
Percentage complete: 97.74
Percentage complete: 97.75
Percentage complete: 97.75
Percentage complete: 97.76
Percentage complete: 97.76
Percentage complete: 97.76
Percentage complete: 97.77
Percentage complete: 97.77
Percentage complete: 97.78
Percentage complete: 97.78
Percentage complete: 97.78
Percentage complete: 97.79
Percentage complete: 97.79
Percentage complete: 97.79
Percentage complete: 97.80
Percentage complete: 97.80
Percentage complete: 97.81
Percentage complete: 97.81
Percentage complete: 97.81
Percentage complete: 97.82
Percentage complete: 97.82
Percentage complete: 97.83
Percentage complete: 97.83
Percentage complete: 97.83
Percentage complete: 97.84
Percentage complete: 97.84
Percentage complete: 97.84
Percentage complete: 97.85
P

Percentage complete: 98.88
Percentage complete: 98.88
Percentage complete: 98.89
Percentage complete: 98.89
Percentage complete: 98.90
Percentage complete: 98.90
Percentage complete: 98.90
Percentage complete: 98.91
Percentage complete: 98.91
Percentage complete: 98.91
Percentage complete: 98.92
Percentage complete: 98.92
Percentage complete: 98.93
Percentage complete: 98.93
Percentage complete: 98.93
Percentage complete: 98.94
Percentage complete: 98.94
Percentage complete: 98.95
Percentage complete: 98.95
Percentage complete: 98.95
Percentage complete: 98.96
Percentage complete: 98.96
Percentage complete: 98.96
Percentage complete: 98.97
Percentage complete: 98.97
Percentage complete: 98.98
Percentage complete: 98.98
Percentage complete: 98.98
Percentage complete: 98.99
Percentage complete: 98.99
Percentage complete: 99.00
Percentage complete: 99.00
Percentage complete: 99.00
Percentage complete: 99.01
Percentage complete: 99.01
Percentage complete: 99.01
Percentage complete: 99.02
P

In [4]:
class TwitterClient(object):
    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
 
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
        
    def get_tweets(self, query, since, until, count):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
        
        tweetCriteria = GetOldTweets.got.manager.TweetCriteria().setQuerySearch(query).setSince(since).setUntil(until).setMaxTweets(count)
        fetched_tweets = GetOldTweets.got.manager.TweetManager.getTweets(tweetCriteria)
        
        # parsing tweets one by one
        for tweet in fetched_tweets:
            # empty dictionary to store required params of a tweet
            parsed_tweet = {}
 
            # saving text of tweet
            parsed_tweet['text'] = tweet.text
            # saving sentiment of tweet
            parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
 
            # appending parsed tweet to tweets list
            if tweet.retweets > 0:
                # if tweet has retweets, ensure that it is appended only once
                if parsed_tweet not in tweets:
                    tweets.append(parsed_tweet)
            else:
                tweets.append(parsed_tweet)
        # return parsed tweets
        return tweets

In [5]:
def get_sentiment(tweets):
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    pos_tweet_percentage = 100*len(ptweets)/len(tweets)
    # print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    neg_tweet_percentage = 100*len(ntweets)/len(tweets)
    # print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    # percentage of neutral tweets
    neutral_tweet_percentage = 100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)
    # print("Neutral tweets percentage: {} %".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))
 
    # printing first 5 positive tweets
    # print("\n\nPositive tweets:")
    # for tweet in ptweets[:10]:
    #    print(tweet['text'])
 
    # printing first 5 negative tweets
    # print("\n\nNegative tweets:")
    # for tweet in ntweets[:10]:
    #    print(tweet['text'])
    return pos_tweet_percentage / 100., neg_tweet_percentage / 100., neutral_tweet_percentage / 100.

In [7]:
# Convert to csv after completed
search_params.to_csv("Twitter_NLP.csv")